In [6]:
using Match

# Virtual Machine 

## AST walking 

In [203]:
struct Atom{T}
    val::T
end
function val(a::Atom)
    return a.val
end
struct SExpr
    expr::Tuple
end
struct Macro{T}
    func::T
end
function (m::Macro)(args...)
    return m.func(args...)
end
function atomtype(a::Atom{T}) where T
    return T
end
function se(args...)
    return SExpr((x -> x isa Atom || x isa SExpr ? x : Atom(x)).(args))
end
function at(arg)
    return Atom(arg)
end

at (generic function with 1 method)

In [119]:
((x...) -> sum(x))(3,4)

7

In [134]:
Dict(zip((:a, :b, :c), (1,2,3)))[:c]

3

In [417]:
BUILTINS = Dict(
    :+ => (+), 
    :* => (*),
    :list => (args...) -> SExpr(args),
    :lambda => :lambda,
    :macro  => :macro,
    :quote => :quote,
)
DEFAULT_ENV = name -> name in keys(BUILTINS) ? BUILTINS[name] : error("Error: $(name) not defined")

function create_lambda(ast, env)
    return (args...) -> ast_walk(
            ast.expr[3],
            env = begin
                nameargs = Dict(zip(val.(ast.expr[2].expr), args))
                name -> name in keys(nameargs) ? nameargs[name] : env(name) 
            end
        )
end

# function ast_walk(ast::SExpr; env=DEFAULT_ENV)
#     temp = ast_walk(ast.expr[1], env=env)
#     @match temp begin
#         :lambda => create_lambda(ast, env)
#         :macro => Macro(create_lambda(ast, env))
#         :quote => ast.expr[2]
#         func::Function => temp(ast_walk.(ast.expr[2:end], env=env)...)
#         macr::Macro    => ast_walk(temp(ast.expr[2:end]...), env=env)
#         _ => error("Not callable")
#     end
# end
function ast_walk(ast::SExpr; env=DEFAULT_ENV)
    @match ast.expr[1] begin
        at(:lambda) => create_lambda(ast, env)
        at(:macro)  => Macro(create_lambda(ast, env))
        at(:quote)  => ast.expr[2]
        _ => begin
            item = ast_walk(ast.expr[1], env=env)
            @match item begin
                _::Function => item(ast_walk.(ast.expr[2:end], env=env)...)
                _::Macro    => ast_walk(item(ast.expr[2:end]...), env=env)
                _ => error("Error: not callable")
            end
        end
    end
end
function ast_walk(ast::Atom; env=DEFAULT_ENV)
    return atomtype(ast) == Symbol ? env(ast.val) : ast.val
end

ast_walk (generic function with 3 methods)

In [418]:
ast_walk(se(:+, se(:*, 3, 4), se(:*, 3, 3)))

21

In [419]:
ast_walk(se(se(:lambda, se(:x,), se(:+, se(:*, :x, 2), 1)), 3))

7

In [420]:
ast_walk(se(se(:macro, se(:x), se(:list, se(:quote, :+), :x, :x)), 4))

8

In [421]:
ast_walk(se(:lambda, se(:let), se(:let, :x, 3, se(:+, :x, 4))))

#723 (generic function with 1 method)

In [422]:
ast_walk(se(:macro, se(:name, :value, :body), se(:list, se(:list, :lambda, se(:name), :body),  :value)))

Macro{var"#723#725"{SExpr, var"#721#722"}}(var"#723#725"{SExpr, var"#721#722"}(SExpr((Atom{Symbol}(:macro), SExpr((Atom{Symbol}(:name), Atom{Symbol}(:value), Atom{Symbol}(:body))), SExpr((Atom{Symbol}(:list), SExpr((Atom{Symbol}(:list), Atom{Symbol}(:lambda), SExpr((Atom{Symbol}(:name),)), Atom{Symbol}(:body))), Atom{Symbol}(:value))))), var"#721#722"()))

In [423]:
ast_walk(se(:quote, se(:x, :y)))

SExpr((Atom{Symbol}(:x), Atom{Symbol}(:y)))

In [424]:
ast_walk(se(:quote, se(:lambda, se(:x), se(:+, :x, 3))))

SExpr((Atom{Symbol}(:lambda), SExpr((Atom{Symbol}(:x),)), SExpr((Atom{Symbol}(:+), Atom{Symbol}(:x), Atom{Int64}(3)))))

In [425]:
ast_walk(
    se(se(:lambda, se(:let), se(:let, :x, 3, se(:*, :x, 4))),
       se(:macro, se(:name, :value, :body), se(:list, se(:list, se(:quote, :lambda), se(:list, :name), :body),  :value)))
)

12

In [426]:
ast_walk(
    se(se(:lambda, se(:let), se(:let, :x, 3, se(:+, :x, 4))),
       se(:macro, se(:name, :value, :body), se(:list, se(:list, se(:quote, :lambda), se(:list, :name), :body),  :value)))
)

7

## Simple Stack Machine

Translation as code execution.
